In [6]:
import os

def readlines(filename):
    """Read all the lines in a text file and return as a list
    """
    with open(filename, 'r') as f:
        lines = f.read().splitlines()
    return lines

fpath = "./splits/endovis/train.txt"
train_filenames = readlines(fpath)
filenames = train_filenames[0]
line = filenames.split()

def get_image_path(folder, frame_index):
    f_str = "{:010d}.png".format(frame_index)
    image_path = os.path.join(folder, "image_02/", f_str)
    return image_path

folder = line[0]
frame_index = int(line[1])
tar_img_path = get_image_path(folder, frame_index)
tar_img_path

'dataset2/keyframe2/image_02/0000000626.png'